<a href="https://colab.research.google.com/github/fabriziobasso/Colab_backup/blob/main/EDA_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **S4E12 - Insurance Premium Prediction Dataset**

## Problem Statement

The goal of this dataset is to facilitate the development and testing of regression models for predicting insurance premiums based on various customer characteristics and policy details. Insurance companies often rely on data-driven approaches to estimate premiums, taking into account factors such as age, income, health status, and claim history. This synthetic dataset simulates real-world scenarios to help practitioners practice feature engineering, data cleaning, and model training.

## Dataset Overview

This dataset contains 2Lk+ and 20 features with a mix of categorical, numerical, and text data. It includes missing values, incorrect data types, and skewed distributions to mimic the complexities faced in real-world datasets. The target variable for prediction is the "Premium Amount".

### Features

1. Age: Age of the insured individual (Numerical)
2. Gender: Gender of the insured individual (Categorical: Male, Female)
3. Annual Income: Annual income of the insured individual (Numerical, skewed)
4. Marital Status: Marital status of the insured individual (Categorical: Single, Married, Divorced)
5. Number of Dependents: Number of dependents (Numerical, with missing values)
6. Education Level: Highest education level attained (Categorical: High School, Bachelor's, Master's, PhD)
7. Occupation: Occupation of the insured individual (Categorical: Employed, Self-Employed, Unemployed)
8. Health Score: A score representing the health status (Numerical, skewed)
9. Location: Type of location (Categorical: Urban, Suburban, Rural)
10. Policy Type: Type of insurance policy (Categorical: Basic, Comprehensive, Premium)
11. Previous Claims: Number of previous claims made (Numerical, with outliers)
12. Vehicle Age: Age of the vehicle insured (Numerical)
13. Credit Score: Credit score of the insured individual (Numerical, with missing values)
14. Insurance Duration: Duration of the insurance policy (Numerical, in years)
15. Premium Amount: Target variable representing the insurance premium amount (Numerical, skewed)
16. Policy Start Date: Start date of the insurance policy (Text, improperly formatted)
17. Customer Feedback: Short feedback comments from customers (Text)
18. Smoking Status: Smoking status of the insured individual (Categorical: Yes, No)
19. Exercise Frequency: Frequency of exercise (Categorical: Daily, Weekly, Monthly, Rarely)
20. Property Type: Type of property owned (Categorical: House, Apartment, Condo)

# Data Characteristics

- Missing Values: Certain features contain missing values to simulate real-world data collection issues.
- Incorrect Data Types: Some fields are intentionally set to incorrect data types to practice data cleaning.
- Skewed Distributions: Numerical features like **Annual Income** and **Premium Amount** have skewed distributions, which can be addressed through transformations.

In [ ]:
%%capture
!pip install -qq pytorch_tabnet
!pip install optuna
!pip install catboost
!pip install optuna-integration-pytorch-tabnet

from pytorch_tabnet.tab_model import TabNetRegressor

!pip install category-encoders
!pip install optuna-integration

!pip install keras-tuner --upgrade
!pip install keras-nlp
!pip install BorutaShap
!pip install scikit-lego
!pip install skops

from pytorch_tabnet.tab_model import TabNetRegressor

In [ ]:
# Setup notebook
from pathlib import Path
import ipywidgets as widgets
import pandas as pd
import numpy as np
from pickle import load, dump
import json
import joblib
#import calplot as cal

# Graphic Libraries:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
# Set Style
sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5});
sns.despine(left=True, bottom=True, top=False, right=False);
mpl.rcParams['figure.dpi'] = 120;
mpl.rc('axes', labelsize=12);
plt.rc('xtick',labelsize=10);
plt.rc('ytick',labelsize=10);

mpl.rcParams['axes.spines.top'] = False;
mpl.rcParams['axes.spines.right'] = False;
mpl.rcParams['axes.spines.left'] = True;

# Palette Setup
colors = ['#FB5B68','#FFEB48','#2676A1','#FFBDB0',]
colormap_0 = mpl.colors.LinearSegmentedColormap.from_list("",colors)
palette_1 = sns.color_palette("coolwarm", as_cmap=True)
palette_2 = sns.color_palette("YlOrBr", as_cmap=True)
palette_3 = sns.light_palette("red", as_cmap=True)
palette_4 = sns.color_palette("viridis", as_cmap=True)
palette_5 = sns.color_palette("rocket", as_cmap=True)
palette_6 = sns.color_palette("GnBu", as_cmap=True)
palette_7 = sns.color_palette("tab20c", as_cmap=False)
palette_8 = sns.color_palette("Set2", as_cmap=False)

palette_custom = ['#fbb4ae','#b3cde3','#ccebc5','#decbe4','#fed9a6','#ffffcc','#e5d8bd','#fddaec','#f2f2f2']
palette_9 = sns.color_palette(palette_custom, as_cmap=False)

# tool for Excel:
from openpyxl import load_workbook, Workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.worksheet.cell_range import CellRange

from openpyxl.formatting import Rule
from openpyxl.styles import Font, PatternFill, Border
from openpyxl.styles.differential import DifferentialStyle

# Bloomberg
#from xbbg import blp
from catboost import CatBoostRegressor, Pool, CatBoostClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from xgboost.callback import EarlyStopping

import lightgbm as lgb
from lightgbm import (LGBMRegressor,
                      LGBMClassifier,
                      early_stopping,
                      record_evaluation,
                      log_evaluation)

# Time Management
from tqdm import tqdm
from datetime import date
from datetime import datetime
from pandas.tseries.offsets import BMonthEnd, QuarterEnd
import datetime
from pandas.tseries.offsets import BDay # BDay is business day, not birthday...
import datetime as dt
import click
import glob
import os
import gc
import re
import string

from ipywidgets import AppLayout
from ipywidgets import Dropdown, Layout, HTML, AppLayout, VBox, Label, HBox, BoundedFloatText, interact, Output

#from my_func import *

import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

from keras.layers import Input, LSTM, Dense, Lambda, RepeatVector, Reshape
from keras.models import Model
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

from keras.utils import FeatureSpace, plot_model

# Import libraries for Hypertuning
import kerastuner as kt
from kerastuner.tuners import RandomSearch, GridSearch, BayesianOptimization

#from my_func import *

# preprocessing modules
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedKFold, cross_val_score, cross_validate, GroupKFold, GridSearchCV, RepeatedStratifiedKFold, cross_val_predict

from sklearn.preprocessing import (LabelEncoder,
                                   StandardScaler,
                                   MinMaxScaler,
                                   OrdinalEncoder,
                                   RobustScaler,
                                   PowerTransformer,
                                   OneHotEncoder,
                                   QuantileTransformer,
                                   PolynomialFeatures)

# metrics
import sklearn
import skops.io as sio
from sklearn.metrics import (mean_squared_error,
                             root_mean_squared_error,
                             root_mean_squared_log_error,
                             r2_score,
                             mean_absolute_error,
                             mean_absolute_percentage_error,
                             classification_report,
                             confusion_matrix,
                             ConfusionMatrixDisplay,
                             multilabel_confusion_matrix,
                             accuracy_score,
                             roc_auc_score,
                             auc,
                             roc_curve,
                             log_loss,
                             make_scorer)
# modeling algos
from sklearn.linear_model import (LogisticRegression,
                                  Lasso,
                                  ridge_regression,
                                  LinearRegression,
                                  Ridge,
                                  RidgeCV,
                                  ElasticNet,
                                  BayesianRidge,
                                  HuberRegressor,
                                  TweedieRegressor,
                                  QuantileRegressor,
                                  ARDRegression,
                                  TheilSenRegressor,
                                  PoissonRegressor,
                                  GammaRegressor)

from sklearn.ensemble import (AdaBoostRegressor,
                              AdaBoostClassifier,
                              RandomForestRegressor,
                              RandomForestClassifier,
                              VotingRegressor,
                              GradientBoostingRegressor,
                              GradientBoostingClassifier,
                              StackingRegressor,
                              StackingClassifier,
                              HistGradientBoostingClassifier,
                              HistGradientBoostingRegressor,
                              ExtraTreesClassifier)

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import FunctionTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

from sklearn.multioutput import RegressorChain
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

import itertools
import warnings
from openpyxl import load_workbook

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import statsmodels.api as sm
from pylab import rcParams
import scipy.stats as ss

warnings.filterwarnings('ignore')
#plt.style.use('fivethirtyeight')

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
<ipython-input-2-b603d7214877>:112: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


<Figure size 640x480 with 0 Axes>

In [ ]:
sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.5,
         'grid.alpha'           :0.5,
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : 'black',
         'axes.titlesize'       : 12,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,
        });

sns.set_style("whitegrid",{"grid.linestyle":"--", 'grid.linewidth':0.2, 'grid.alpha':0.5})
# Set Style
mpl.rcParams['figure.dpi'] = 120;

# Making sklearn pipeline outputs as dataframe:-
pd.set_option('display.max_columns', 100);
pd.set_option('display.max_rows', 50);

sns.despine(left=True, bottom=True, top=False, right=False)

mpl.rcParams['axes.spines.left'] = True
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.bottom'] = True

<Figure size 960x660 with 0 Axes>

In [ ]:
from tqdm import tqdm
from itertools import product

import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import GroupKFold
from sklearn.impute import SimpleImputer
import torch

import warnings
warnings.filterwarnings("ignore")

# Connect to Colab:#
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


<div style="text-align:center; border-radius:15px; padding:15px; margin:0; font-size:100%; font-family:Arial, sans-serif; background-color:#A8DADC; color:#1D3557; overflow:hidden; box-shadow:0 3px 6px rgba(0, 0, 0, 0.2);">
    <h3>Loading and Preprocessing Data for Compatibility</h3>
</div>


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/train_no_nan.csv')

df_test = pd.read_csv('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/test_no_nan.csv')

df_test_orig = pd.read_csv(
    '/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/test.csv',
     index_col='id',
     parse_dates=['Policy Start Date'],
)

df_train_orig = pd.read_csv(
    '/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/train.csv',
     index_col='id',
     parse_dates=['Policy Start Date'],
)

df_subm = pd.read_csv(
    "/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/sample_submission.csv",
     index_col='id',
)

# df_orig = pd.read_csv(
#     "/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/Insurance Premium Prediction Dataset.csv",
#      parse_dates=['Policy Start Date'],
#     #     index_col='id',
# )


mapping = {2.0:0.0,0.0:1.0,1.0:2.0}
df_train["Customer Feedback"] = df_train["Customer Feedback"].map(mapping)
df_test["Customer Feedback"] = df_test["Customer Feedback"].map(mapping)

In [ ]:
# # Convert `Policy Start Date` column to datetime64 format
# df_orig['Policy Start Date'] = pd.to_datetime(df_orig['Policy Start Date'])

# # Calculate the difference in days between today and the `Policy Start Date` column
# today = pd.to_datetime('today')
# difference_in_days = today - df_orig['Policy Start Date']

# # Divide the difference in days by 365 to get the difference in years
# difference_in_years = difference_in_days / pd.Timedelta(days=365)

# # Convert the `Policy Start Date` column to the number of years since the policy start date
# df_orig['Policy Start Date'] = difference_in_years

In [ ]:
df_train["Start_Year"] = df_train_orig["Policy Start Date"].dt.year
df_train["Start_Month"] = df_train_orig["Policy Start Date"].dt.month
df_train["Start_Day"] = df_train_orig["Policy Start Date"].dt.day

df_test["Start_Year"] = df_test_orig["Policy Start Date"].dt.year.values
df_test["Start_Month"] = df_test_orig["Policy Start Date"].dt.month.values
df_test["Start_Day"] = df_test_orig["Policy Start Date"].dt.day.values

In [ ]:
# df_orig.dropna(axis=0,how="any").shape
df_train_orig.head()
#df_test["Policy Start Date"].dt.year

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
id,,,,,,,,,,,,,,,,,,,,
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [ ]:
df_train.shape, df_test.shape, df_subm.shape #, df_orig.shape, df_orig.shape

((1200000, 23), (800000, 23), (800000, 1))

In [ ]:
print("Pytorch Version: {}".format(torch.__version__))
print("SKLearn Version: {}".format(sklearn.__version__))

Pytorch Version: 2.5.1+cu121
SKLearn Version: 1.5.2


In [ ]:
100*df_train.isnull().sum()/df_train.shape[0]

,0
Age,0.0
Gender,0.0
Annual Income,0.0
Marital Status,0.0
Number of Dependents,0.0
Education Level,0.0
Occupation,0.0
Health Score,0.0
Location,0.0
Policy Type,0.0


In [ ]:
100*df_test.isnull().sum()/df_test.shape[0]

,0
Age,0.0
Gender,0.0
Annual Income,0.0
Marital Status,0.0
Number of Dependents,0.0
Education Level,0.0
Occupation,0.0
Health Score,0.0
Location,0.0
Policy Type,0.0


In [ ]:
df_train.head(3)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 23 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Age                   1200000 non-null  float64
 1   Gender                1200000 non-null  object 
 2   Annual Income         1200000 non-null  float64
 3   Marital Status        1200000 non-null  float64
 4   Number of Dependents  1200000 non-null  float64
 5   Education Level       1200000 non-null  object 
 6   Occupation            1200000 non-null  float64
 7   Health Score          1200000 non-null  float64
 8   Location              1200000 non-null  object 
 9   Policy Type           1200000 non-null  object 
 10  Previous Claims       1200000 non-null  float64
 11  Vehicle Age           1200000 non-null  float64
 12  Credit Score          1200000 non-null  float64
 13  Insurance Duration    1200000 non-null  float64
 14  Policy Start Date     1200000 non-

In [ ]:
df_train.nunique()

,0
Age,47
Gender,2
Annual Income,91633
Marital Status,3
Number of Dependents,5
Education Level,4
Occupation,3
Health Score,532670
Location,3
Policy Type,3


In [ ]:
categorical_cols = ["Gender","Marital Status","Number of Dependents","Education Level","Occupation","Location","Policy Type","Previous Claims","Insurance Duration","Customer Feedback","Smoking Status","Exercise Frequency","Property Type",
                    "Start_Year","Start_Month","Start_Day"]
numerical_cols = ["Age","Annual Income","Health Score","Vehicle Age","Credit Score","Policy Start Date"]

target = ['Premium Amount']

len(categorical_cols+numerical_cols+target),len(df_train.columns)

df_train[categorical_cols] = df_train[categorical_cols].astype('category')
df_test[categorical_cols] = df_test[categorical_cols].astype('category')

df_train[numerical_cols] = df_train[numerical_cols].astype(np.float64)
df_test[numerical_cols] = df_test[numerical_cols].astype(np.float64)

## 1.0 Time Features:

Here the following features are reviewed:

* "Start_Year"
* "Start_Month"
* "Start_Day"
* "Policy Start Date"

### **Year**

In [ ]:
# ts_monthly = df_train.groupby(["Start_Year"], as_index=False)["Premium Amount"].agg(["mean","std","skew","median","min","max","count"])
# ts_monthly_test = df_test.groupby(["Start_Year"], as_index=False)["Health Score"].agg(["mean","std","skew","min","max","count"])

# fig, axs = plt.subplots(1,2,figsize=(10, 4))
# ts_monthly["mean"].plot(color="#0485d1", ax=axs[0])
# ts_monthly["std"].plot(color="#c875c4", ax=axs[0])
# ts_monthly["median"].plot(color="#fd411e", ax=axs[0])
# ts_monthly["skew"].plot(color="#fd411e", ax=axs[1])

### **Month**

In [ ]:
# ts_monthly = df_train.groupby(["Start_Month"], as_index=False)["Premium Amount"].agg(["mean","std","skew","median","min","max","count"])
# ts_monthly_test = df_test.groupby(["Start_Month"], as_index=False)["Health Score"].agg(["mean","std","skew","min","max","count"])

# fig, axs = plt.subplots(1,2,figsize=(10, 4))
# ts_monthly["mean"].plot(color="#0485d1", ax=axs[0])

# ts_monthly["skew"].plot(color="#fd411e", ax=axs[1])

### **Year and Month**

In [ ]:
ts_monthly = df_train.groupby(["Start_Year","Start_Month"])["Premium Amount"].agg(["mean","std","skew","median","min","max","count"])
ts_monthly_test = df_test.groupby(["Start_Year","Start_Month"], as_index=False)["Health Score"].agg(["mean","std","skew","min","max","count"])

fig, axs = plt.subplots(1,2,figsize=(10, 4))
ts_monthly["mean"].plot(color="#0485d1", ax=axs[0])
ts_monthly["std"].plot(color="#c875c4", ax=axs[0])
ts_monthly["median"].plot(color="#fd411e", ax=axs[0])
ts_monthly["skew"].plot(color="#fd411e", ax=axs[1])
plt.show()

In [ ]:
ts_monthly=ts_monthly.dropna(axis=0,how="any")
ts_monthly_test=ts_monthly_test.dropna(axis=0,how="any")
ts_monthly.head()

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(13, 4))
ts_monthly["count"].plot(kind="bar",color="#1fa774", ax=axs[0])
ts_monthly_test["count"].plot(kind="bar",color="#fd411e", ax=axs[1])
plt.show()

In [ ]:
ts_day = df_train.groupby(["Start_Day"], as_index=False)["Premium Amount"].agg(["mean","std","skew","median","min","max","count"])
ts_day_test = df_test.groupby(["Start_Day"], as_index=False)["Health Score"].agg(["mean","std","skew","min","max","count"])

fig, axs = plt.subplots(1,2,figsize=(10, 4))
ts_day["mean"].plot(color="#0485d1", ax=axs[0])
ts_day["std"].plot(color="#c875c4", ax=axs[0])
ts_day["median"].plot(color="#fd411e", ax=axs[0])
ts_day["skew"].plot(color="#fd411e", ax=axs[1])

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10, 4))
ts_day["count"].plot(kind="bar",color="#1fa774", ax=axs[0])
ts_day_test["count"].plot(kind="bar",color="#fd411e", ax=axs[1])

In [ ]:
ts_monthly_test

### **Year and Month + Policy Type**

In [ ]:
ts_monthly = df_train.groupby(["Start_Year","Start_Month","Policy Type"], as_index=False)[["Premium Amount"]].agg(["mean"])
# fig, axs = plt.subplots(1,2,figsize=(10, 4))
# ts_monthly[("Premium Amount","mean")].plot(ax=axs[0])
# ts_monthly[("Credit Score","mean")].plot(ax=axs[1])
# ts_monthly["std"].plot(color="#c875c4", ax=axs[0])
# ts_monthly["median"].plot(color="#fd411e", ax=axs[0])
# ts_monthly["skew"].plot(color="#fd411e", ax=axs[1])

In [ ]:
ts_monthly[ts_monthly["Policy Type"]=="Basic"][("Premium Amount","mean")].plot(color="#c875c4")
ts_monthly[ts_monthly["Policy Type"]=="Comprehensive"][("Premium Amount","mean")].plot(color="#fd411e")
ts_monthly[ts_monthly["Policy Type"]=="Premium"][("Premium Amount","mean")].plot(color="#0485d1")

In [ ]:
ts_monthly

### **Year and Month + Customer Feedback + Policy Type**

In [ ]:
ts_monthly = df_train.groupby(["Start_Year","Start_Month","Customer Feedback","Policy Type"], as_index=False)[["Premium Amount"]].agg(["mean"])
# fig, axs = plt.subplots(1,2,figsize=(10, 4))
# ts_monthly[("Premium Amount","mean")].plot(ax=axs[0])
# ts_monthly[("Credit Score","mean")].plot(ax=axs[1])
# ts_monthly["std"].plot(color="#c875c4", ax=axs[0])
# ts_monthly["median"].plot(color="#fd411e", ax=axs[0])
# ts_monthly["skew"].plot(color="#fd411e", ax=axs[1])
ts_monthly.head()

In [ ]:
ts_monthly[(ts_monthly["Customer Feedback"]==0)&(ts_monthly["Policy Type"]=="Premium")][("Premium Amount","mean")].plot(color="#c875c4")
ts_monthly[(ts_monthly["Customer Feedback"]==1)&(ts_monthly["Policy Type"]=="Premium")][("Premium Amount","mean")].plot(color="#fd411e")
ts_monthly[(ts_monthly["Customer Feedback"]==2)&(ts_monthly["Policy Type"]=="Premium")][("Premium Amount","mean")].plot(color="#0485d1")

In [ ]:
ts_monthly[(ts_monthly["Customer Feedback"]==0)&(ts_monthly["Policy Type"]=="Comprehensive")][("Premium Amount","mean")].plot(color="#c875c4")
ts_monthly[(ts_monthly["Customer Feedback"]==1)&(ts_monthly["Policy Type"]=="Comprehensive")][("Premium Amount","mean")].plot(color="#fd411e")
ts_monthly[(ts_monthly["Customer Feedback"]==2)&(ts_monthly["Policy Type"]=="Comprehensive")][("Premium Amount","mean")].plot(color="#0485d1")

In [ ]:
ts_monthly[(ts_monthly["Customer Feedback"]==0)&(ts_monthly["Policy Type"]=="Basic")][("Premium Amount","mean")].plot(color="#c875c4")
ts_monthly[(ts_monthly["Customer Feedback"]==1)&(ts_monthly["Policy Type"]=="Basic")][("Premium Amount","mean")].plot(color="#fd411e")
ts_monthly[(ts_monthly["Customer Feedback"]==2)&(ts_monthly["Policy Type"]=="Basic")][("Premium Amount","mean")].plot(color="#0485d1")

#### Add Mean to Dataset:

In [ ]:
ts_monthly.head(3)
ts_monthly = ts_monthly.droplevel(level=1,axis=1).rename(columns={'Premium Amount': 'Premium_time_Mean'})

In [ ]:
ts_monthly.head()

In [ ]:
df_train_new = pd.merge(df_train, ts_monthly, on=["Start_Year","Start_Month","Customer Feedback","Policy Type"], how='left')
df_test_new = pd.merge(df_test, ts_monthly, on=["Start_Year","Start_Month","Customer Feedback","Policy Type"], how='left')

In [ ]:
df_train_new.isna().sum(),df_test_new.isna().sum()

In [ ]:
plt.scatter(df_train_new["Premium_time_Mean"],df_train_new["Premium Amount"])
plt.show()

In [ ]:
df_train_new.head()

### **Year and Month + Customer Feedback + Policy Type**

In [ ]:
ts_monthly = df_train.groupby(["Start_Year","Start_Month","Smoking Status","Policy Type"], as_index=False)[["Premium Amount"]].agg(["mean"])
# fig, axs = plt.subplots(1,2,figsize=(10, 4))
# ts_monthly[("Premium Amount","mean")].plot(ax=axs[0])
# ts_monthly[("Credit Score","mean")].plot(ax=axs[1])
# ts_monthly["std"].plot(color="#c875c4", ax=axs[0])
# ts_monthly["median"].plot(color="#fd411e", ax=axs[0])
# ts_monthly["skew"].plot(color="#fd411e", ax=axs[1])
ts_monthly.head()

In [ ]:
ts_monthly[(ts_monthly["Smoking Status"]=="No")&(ts_monthly["Policy Type"]=="Comprehensive")][("Premium Amount","mean")].plot(color="#fd411e")
ts_monthly[(ts_monthly["Smoking Status"]=="Yes")&(ts_monthly["Policy Type"]=="Comprehensive")][("Premium Amount","mean")].plot(color="#0485d1")

## Previous Claims and Policy Type:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

plt_1 = sns.boxplot(data=df_train_new, x='Previous Claims', y='Premium Amount', ax=ax[0], palette=palette_9)
plt_2 = sns.boxplot(data=df_train_new, x='Policy Type', y='Premium Amount', ax=ax[1], palette=palette_9);

In [ ]:
df_train_new[df_train_new["Previous Claims"]==9]

In [ ]:
df_test_new[df_test_new["Previous Claims"]==9]

In [ ]:
df_train_new[df_train_new["Previous Claims"]==8].shape, df_test_new[df_test_new["Previous Claims"]==8].shape

## Customer Feedback and Insurance Duration:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

plt_1 = sns.boxplot(data=df_train_new, x='Customer Feedback', y='Premium Amount', ax=ax[0], palette=palette_9)
plt_2 = sns.boxplot(data=df_train_new, x='Insurance Duration', y='Premium Amount', ax=ax[1], palette=palette_9);

## Annual Income and Credit Score:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

plt_1 = sns.scatterplot(data=df_train_new, x='Annual Income', y='Premium Amount',ax=ax[0], palette=palette_9)
plt_2 = sns.boxplot(data=df_train_new, x='Property Type', y='Premium Amount', ax=ax[1], palette=palette_9);

#### **Number of Dependents and Marital Status**

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

plt_1 = sns.boxplot(data=df_train_new, x='Number of Dependents', y='Premium Amount',ax=ax[0], palette=palette_9)
plt_2 = sns.boxplot(data=df_train_new, x='Marital Status', y='Premium Amount', ax=ax[1], palette=palette_9);

In [ ]:
df_train_new["Property Type"].nunique(),df_train_new["Property Type"].unique()

In [ ]:
def replace_entries(df):

  map_gender = {'Male': 0, 'Female': 1}
  df['Gender'] = df['Gender'].map(map_gender)

  map_education = {'High School': 0, "Bachelor's": 1, "Master's": 2, 'PhD': 3}
  df['Education Level'] = df['Education Level'].map(map_education)

  map_loc = {'Rural': 0, 'Suburban': 1, 'Urban': 2}
  df['Location'] = df['Location'].map(map_loc)

  map_policy = {'Basic': 0, 'Comprehensive': 1, 'Premium': 2}
  df['Policy Type'] = df['Policy Type'].map(map_policy)

  map_smoking = {'No': 0, 'Yes': 1}
  df['Smoking Status'] = df['Smoking Status'].map(map_smoking)

  map_exercise = {'Rarely': 0, 'Monthly': 1, 'Weekly': 2, 'Daily': 3}
  df['Exercise Frequency'] = df['Exercise Frequency'].map(map_exercise)

  map_property = {'Condo': 0, 'Apartment': 1, 'House': 2}
  df['Property Type'] = df['Property Type'].map(map_property)

  return df

df_train_new = replace_entries(df_train_new)
df_test_new = replace_entries(df_test_new)


In [ ]:
df_train_new.head()

In [ ]:
df_train_new.describe(include="all").T
#df_train_new.info()

In [ ]:
cat_cols = ['Gender', 'Marital Status','Number of Dependents','Occupation','Location','Policy Type','Previous Claims', 'Insurance Duration', 'Customer Feedback',
            'Smoking Status',  'Exercise Frequency', 'Property Type', 'Start_Year', 'Start_Month', 'Start_Day','Education Level']

num_cols = ['Age', 'Annual Income', 'Health Score', 'Vehicle Age', 'Credit Score', 'Policy Start Date', 'Premium Amount', 'Premium_time_Mean']

dtypes_num = {c:"float" for c in num_cols}
dtypes_cat = {c:"category" for c in cat_cols}

dtypes_all = {**dtypes_num, **dtypes_cat}

len(cat_cols+num_cols),len(dtypes_all.keys())#,len(df_train_new.columns)

In [ ]:
#df_train_new.to_csv('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/df_train_v3.csv', index=False)
#df_test_new.to_csv('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/df_test_v3.csv', index=False)

df_train_new = pd.read_csv('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/df_train_v3.csv', dtype=dtypes_all)
df_test_new = pd.read_csv('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/df_test_v3.csv', dtype=dtypes_all)

In [ ]:
df_train_new.info()

In [ ]:
df_test_new.info()

## **MODELS**

**DATA UPLOAD**

In [ ]:
cat_cols = ['Gender', 'Marital Status','Number of Dependents','Occupation','Location','Policy Type','Previous Claims', 'Insurance Duration', 'Customer Feedback',
            'Smoking Status',  'Exercise Frequency', 'Property Type', 'Start_Year', 'Start_Month', 'Start_Day','Education Level']

num_cols = ['Age', 'Annual Income', 'Health Score', 'Vehicle Age', 'Credit Score', 'Policy Start Date', 'Premium Amount', 'Premium_time_Mean']

dtypes_num = {c:"float" for c in num_cols}
dtypes_cat = {c:"category" for c in cat_cols}

dtypes_all = {**dtypes_num, **dtypes_cat}

len(cat_cols+num_cols),len(dtypes_all.keys())#,len(df_train_new.columns)

(24, 24)

In [ ]:
#df_train_new.to_csv('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/df_train_v3.csv', index=False)
#df_test_new.to_csv('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/df_test_v3.csv', index=False)

df_train_new = pd.read_csv('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/df_train_v3.csv', dtype=dtypes_all)
df_test_new = pd.read_csv('/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/df_test_v3.csv', dtype=dtypes_all)

In [ ]:
df_train_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype   
---  ------                --------------    -----   
 0   Age                   1200000 non-null  float64 
 1   Gender                1200000 non-null  category
 2   Annual Income         1200000 non-null  float64 
 3   Marital Status        1200000 non-null  category
 4   Number of Dependents  1200000 non-null  category
 5   Education Level       1200000 non-null  category
 6   Occupation            1200000 non-null  category
 7   Health Score          1200000 non-null  float64 
 8   Location              1200000 non-null  category
 9   Policy Type           1200000 non-null  category
 10  Previous Claims       1200000 non-null  category
 11  Vehicle Age           1200000 non-null  float64 
 12  Credit Score          1200000 non-null  float64 
 13  Insurance Duration    1200000 non-null  category
 14  Policy Start Date 

**FUNCTIONS**

In [ ]:
cat_cols = ['Gender', 'Marital Status','Number of Dependents','Occupation','Location','Policy Type','Previous Claims', 'Insurance Duration', 'Customer Feedback',
            'Smoking Status',  'Exercise Frequency', 'Property Type', 'Start_Year', 'Start_Month', 'Start_Day','Education Level']

num_cols = ['Age', 'Annual Income', 'Health Score', 'Vehicle Age', 'Credit Score', 'Policy Start Date', 'Premium_time_Mean']

params ={'reg_alpha': 0.4932, 'reg_lambda': 0.002076, 'max_depth': 21, 'colsample_bytree': 0.825, 'subsample': 0.75, 'num_leaves': 101, 'n_estimators': 2501, 'learning_rate': 0.01,
         'min_child_samples':35, 'random_state': 42, 'force_col_wise':True, 'verbose':-1}

In [ ]:
def rmsle_obj(y_true, y_pred):
    grad = -2 * (np.log1p(y_true) - np.log1p(y_pred)) / (y_pred + 1)
    hess = 2 * (np.log1p(y_true) - np.log1p(y_pred) + 1) / ((y_pred + 1) ** 2)
    return grad, hess

def rmsle_objective(y_true, y_pred):
    y_pred = np.log1p(y_pred)  # Apply log1p(x) = log(1 + x) to predicted values
    y_true = np.log1p(y_true)  # Apply log1p to true values
    grad = (y_pred - y_true) / (y_pred * y_true)
    hess = 1 / (y_pred * y_true)
    return grad, hess

def rmsle_metric(y_true, y_pred):
    y_true = np.expm1(y_true)
    y_pred = np.expm1(y_pred)
    rmsle = np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2))
    return 'rmsle', rmsle, False

# Model Training
def train_lgbm(params, X, y, cat_loc, use_gpu=True, X_val=None, y_val=None, es=101):

    # Set to CPU if GPU usage is specified
    if use_gpu:
      params['device'] = 'gpu'
    else:
      params['device'] = 'cpu'

    model = LGBMRegressor(**params, objective="root_mean_squared_error", metric="rmse",boosting_type='gbdt', categorical_feature=cat_loc)
    model.fit(X, y, eval_set=(X_val, y_val), callbacks=[early_stopping(stopping_rounds=es)])

    return model

# Model Training
def train_lgbm_rsmle(params, X, y, cat_loc, use_gpu=True, X_val=None, y_val=None, es=101):

    # Set to CPU if GPU usage is specified
    if use_gpu:
      params['device'] = 'gpu'
    else:
      params['device'] = 'cpu'

    # Remove the 'metric' key from params or set it to a valid string or list of strings.
    params.pop('metric', None)  # Remove if present
    # Or, set to a valid string:
    # params['metric'] = 'rmse'

    model = LGBMRegressor(**params, objective=rmsle_obj,boosting_type='gbdt', categorical_feature=cat_loc)
    model.fit(X, y, eval_set=(X_val, y_val), callbacks=[early_stopping(stopping_rounds=es)], eval_metric=[rmsle_metric])

    return model

In [ ]:
def plot_results(y,y_forecasted):
  fig, axs = plt.subplots(1,2,figsize=(10, 4))

  axs[0].hist(y_forecasted, bins=100, alpha=0.5, color="royalblue")
  axs[1].hist(y, bins=100, color="salmon", alpha=0.5)
  axs[0].set_xlabel("Premium Amount Forecast: Train")
  axs[1].set_xlabel("Premium Amount Forecast: Test")
  axs[0].set_ylabel("Frequency")
  plt.suptitle("Distribution of Predicted Premium Amounts", y=1.01)
  plt.show()

def store_results(for_test,for_train, model="LGBM", experiment=0):
  df_test_for = for_test.copy()
  df_train_for = for_train.copy()

  df_train_for["Average"] = df_train_for.mean(axis=1)

  train_forecast_to_store = df_train_for[["Average"]]
  train_forecast_to_store["Average"] = train_forecast_to_store["Average"].astype("float")
  train_forecast_to_store.columns = [f"{model}_{experiment}"]

  plot_results(train_forecast_to_store,df_test_for)

  print(train_forecast_to_store.min(),train_forecast_to_store.max(),train_forecast_to_store.mean(),train_forecast_to_store.median())

  df_test_for.to_csv(f'/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/Submissions/submission_{model}_{experiment}.csv')
  train_forecast_to_store.to_csv(f'/content/drive/MyDrive/Exercises/Studies_Structured_Data/Data/S4E12/Submissions/train_{model}_{experiment}.csv')

  print(f'Results of {model}_{experiment} all saved')

  return (df_test_for, train_forecast_to_store)

## **1.0 LGBMRegressor** - RMSE on Log Target

In [ ]:
X = df_train_new.drop(columns=["Premium Amount"])
X_test = df_test_new.drop(columns=["Premium Amount"])
y = df_train_new["Premium Amount"]

log_y = np.log1p(y)

In [ ]:
cat_loc = [X.columns.get_loc(i) for i in cat_cols]

### 1.1 Optuna Optimization:

In [ ]:
def objective_lgbm(trial, X, y, n_splits, n_repeats, use_gpu=False):

    model_class = LGBMRegressor

    categorical_features = cat_cols.copy()
    tot_cat = categorical_features

    numeric_features = [col for col in X.columns if col not in tot_cat]

    params = {

    'num_leaves':         101, #trial.suggest_int('num_leaves', 31, 111, step=5),
    'n_estimators':       2501,
    'learning_rate':      0.01,
    'min_child_samples':  35, #trial.suggest_int('min_child_samples', 31, 51, step=1),
    #'min_child_weight' :  trial.suggest_float("min_child_weight", 1e-2, 1.0, log=True),
    "reg_alpha" :         trial.suggest_float("reg_alpha", 1e-3, 1.0, log=True),
    "reg_lambda" :        trial.suggest_float("reg_lambda", 1e-3, 1.0, log=True),
    "max_depth" :         trial.suggest_int('max_depth', 8, 21, step=1),
    'colsample_bytree':   trial.suggest_float("colsample_bytree", 0.65, 0.95, step=0.025),
    'subsample':          trial.suggest_float("subsample", 0.65, 0.95, step=0.025),
    'random_state':       42,
    'force_col_wise':     True,
    'verbose':-1
    }

    kf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)
    rmsle_scores = []

    for idx_train, idx_valid in kf.split(X, y):

        # Split the data into training and validation sets for the current fold
        X_train, y_train = X.iloc[idx_train], y.iloc[idx_train].to_numpy().reshape(-1,1)
        X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid].to_numpy().reshape(-1,1)

        scaler = StandardScaler()
        X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
        X_valid[num_cols] = scaler.transform(X_valid[num_cols])

        X_train = X_train.to_numpy()
        X_valid = X_valid.to_numpy()

        # Create the pipeline
        model = model_class(**params, objective="root_mean_squared_error", metric="rmse",boosting_type='gbdt', categorical_feature=cat_loc)
        # Create the early stopping callback
        early_stop = early_stopping(stopping_rounds=101)
        # Fit the model:
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[early_stop])

        # Make predictions on the validation set
        y_pred = model.predict(X_valid)

        y_pred = np.expm1(y_pred)
        y_valid = np.expm1(y_valid)

        # Calculate the RMSE for the current fold

        rmsle_score = root_mean_squared_log_error(y_valid, y_pred)
        rmsle_scores.append(rmsle_score)

    # Calculate the mean RMSLE score across all folds
    mean_rmsle_score = np.mean(rmsle_scores)

    return mean_rmsle_score

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(X, y, model_class, n_trials, n_splits_ ,n_repeats_, use_gpu=False):  #use_gpu
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
    study.optimize(lambda trial: objective_lgbm(trial, X, y, n_splits=n_splits_, n_repeats=n_repeats_, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

# Step 3: Saving Best Results and Models
def save_results(study, model_class, model_name):
    best_params_file = f"{model_name}_best_params.joblib"
    joblib.dump(study.best_params, best_params_file)
    print(f"Best parameters for {model_name} saved to {best_params_file}")

    verbose_file = f"{model_name}_optuna_verbose.log"
    with open(verbose_file, "w") as f:
        f.write(str(study.trials))
    print(f"Optuna verbose for {model_name} saved to {verbose_file}")

In [ ]:
# usage with XGBRegressor
lgbm_study = tune_hyperparameters(X, log_y, model_class=LGBMRegressor, n_trials=51, n_splits_ = 3 ,n_repeats_=3, use_gpu=False)
save_results(lgbm_study, LGBMRegressor, "LGBMBoost_ext")
lgbm_params = lgbm_study.best_params

In [ ]:
print(lgbm_params)

In [ ]:
trial = lgbm_study.best_trial
print('MSE: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

* Experiment 1:

    - MSE: 1.0685007914142697
    - Best hyperparameters: {'num_leaves': 111, 'min_child_samples': 35, 'reg_alpha': 005448690915044739, 'reg_lambda': 0.016061276667668913, 'max_depth': 12, 'colsample_bytree': 0.8500000000000001, 'subsample': 0.75}

* Experiment 2:

    - MSE: 1.068389091315945
    - Best hyperparameters: {'reg_alpha': 0.4932, 'reg_lambda': 0.002076, 'max_depth': 21, 'colsample_bytree': 0.825, 'subsample': 0.75,  'num_leaves': 101, 'n_estimators': 2501,'learning_rate': 0.01, 'min_child_samples':  35}    

In [ ]:
fig = optuna.visualization.plot_optimization_history(lgbm_study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_param_importances(lgbm_study)
fig.show()

In [ ]:
#del xgb_study
gc.collect()

### 1.2 Fit Best Model:

**PARAMETERS**

In [ ]:
# Define a common random seed for reproducibility
RANDOM_SEED = 42
N_ESTIMATORS = 3000  # Number of estimators for the ensemble models
n_splits = 3
n_repeats = 3

params ={'reg_alpha': 0.4932, 'reg_lambda': 0.002076, 'max_depth': 21, 'colsample_bytree': 0.825, 'subsample': 0.75, 'num_leaves': 101, 'n_estimators': 2501, 'learning_rate': 0.01,
         'min_child_samples':35, 'random_state': 42, 'force_col_wise':True, 'verbose':0}

cat_cols = ['Gender', 'Marital Status','Number of Dependents','Occupation','Location','Policy Type','Previous Claims', 'Insurance Duration', 'Customer Feedback',
            'Smoking Status',  'Exercise Frequency', 'Property Type', 'Start_Year', 'Start_Month', 'Start_Day','Education Level']

num_cols = ['Age', 'Annual Income', 'Health Score', 'Vehicle Age', 'Credit Score', 'Policy Start Date', 'Premium_time_Mean']

cat_loc = [X.columns.get_loc(i) for i in cat_cols]

**DATA**

In [ ]:
df_subm_stack = df_subm.copy()

X = df_train_new.drop(columns=["Premium Amount"])
X_test = df_test_new.drop(columns=["Premium Amount"])
y = df_train_new["Premium Amount"]

log_y = np.log1p(y)

**FIT THE MODEL**

In [ ]:
kf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)
rmsle = []

# Initialize the Stack
df_subm_stack['Premium Amount'] = 0.0

i=0

oof_results_stack = pd.DataFrame(columns=list(range(n_splits*n_repeats)), index=X.index)

for idx_train, idx_valid in kf.split(X, log_y):

    print(f"Working on Fold {i}")

    # Split the data into training and validation sets for the current fold
    X_train, y_train = X.iloc[idx_train], log_y.iloc[idx_train].to_numpy().reshape(-1,1)
    X_valid, y_valid = X.iloc[idx_valid], log_y.iloc[idx_valid].to_numpy().reshape(-1,1)
    X_test_ = X_test.copy()

    scaler = StandardScaler()
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_valid[num_cols] = scaler.transform(X_valid[num_cols])
    X_test_[num_cols] = scaler.transform(X_test[num_cols])


    X_train = X_train.to_numpy()
    X_valid = X_valid.to_numpy()
    X_test_ = X_test_.to_numpy()


    if i >= 9:
        #print(stacking_model.get_params())
        # Fit the StackingRegressor
        LGBM_model = train_lgbm(params, X_train, y_train, cat_loc, use_gpu=False, X_val=X_valid, y_val=y_valid, es=101)
        obj = sio.dump(LGBM_model, f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S4E12/LGBM_base_{i}.skops")

    else:
        unknown_types = sio.get_untrusted_types(file=f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S4E12/LGBM_base_{i}.skops")
        LGBM_model = sio.load(f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S4E12/LGBM_base_{i}.skops", trusted=unknown_types)

    stack_preds = np.exp(LGBM_model.predict(X_valid))

    oof_results_stack.iloc[idx_valid,i] = stack_preds.flatten()
    # Prepare the test data and make predictions
    error = root_mean_squared_log_error(np.exp(y_valid), stack_preds)

    rmsle.append(error)
    print(f"RMSLE fold {i}: {error}")

    # Aggregate the predictions across the 5 folds (averaging for ensemble)
    df_subm_stack['Premium Amount'] += np.exp(LGBM_model.predict(X_test_)) / (n_splits*n_repeats)
    i += 1

In [ ]:
np.mean(rmsle), np.std(rmsle)

### **1.3 Save Results:**

In [ ]:
(df_test_for,train_forecast_to_store) = store_results(for_test=df_subm_stack,for_train=oof_results_stack, model="LGBM", experiment=0)

In [ ]:
df_test_for.max(),train_forecast_to_store.max()

In [ ]:
train_forecast_to_store

## **2.0 CatBoostRegressor**

In [ ]:
X = df_train_new.drop(columns=["Premium Amount"])
X_test = df_test_new.drop(columns=["Premium Amount"])
y = df_train_new["Premium Amount"]

log_y = np.log1p(y)

In [ ]:
cat_loc = [X.columns.get_loc(i) for i in cat_cols]

### 1.1 Optuna Optimization:

In [ ]:
def objective_catboost(trial, X, y, n_splits, n_repeats, model=CatBoostRegressor, use_gpu=True):

    model_class = model

    categorical_features = cat_cols.copy()
    tot_cat = categorical_features

    numeric_features = [col for col in X.columns if col not in tot_cat]

    params = {
        'iterations': 2501,
        'learning_rate': 0.025, #trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 7, 15),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-4, 1.0, log=True),
        "bootstrap_type": "Bernoulli",
        'subsample': trial.suggest_float('subsample ', 0.65, 0.90,step=0.025),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.5),
        #'border_count': trial.suggest_int('border_count', 32, 255),
        'cat_features': categorical_features,
        'task_type': 'GPU' if use_gpu else 'CPU',
        'verbose': 100
    }

    kf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)
    rmsle_scores = []

    for idx_train, idx_valid in kf.split(X, y):

        # Split the data into training and validation sets for the current fold
        X_train, y_train = X.iloc[idx_train], y.iloc[idx_train].to_numpy().reshape(-1,1)
        X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid].to_numpy().reshape(-1,1)

        scaler = StandardScaler()
        X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
        X_valid[num_cols] = scaler.transform(X_valid[num_cols])

        # Create the Pool objects for CatBoost
        train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features)
        valid_pool = Pool(data=X_valid, label=y_valid, cat_features=categorical_features)

        # Create the pipeline
        model = model_class(**params, loss_function='RMSE', eval_metric='RMSE')
        # Fit the model:
        model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=101,
                  #callbacks=[optuna.integration.CatBoostPruningCallback(trial, "RMSE")]
                  )

        # Make predictions on the validation set
        y_pred = model.predict(X_valid)

        y_pred = np.expm1(y_pred)
        y_valid = np.expm1(y_valid)

        # Calculate the RMSE for the current fold

        rmsle_score = root_mean_squared_log_error(y_valid, y_pred)
        rmsle_scores.append(rmsle_score)

    # Calculate the mean RMSLE score across all folds
    mean_rmsle_score = np.mean(rmsle_scores)

    return mean_rmsle_score

In [ ]:
# Step 2: Tuning Hyperparameters with Optuna
def tune_hyperparameters(X, y, model_class, n_trials, n_splits_ ,n_repeats_, use_gpu=True):  #use_gpu
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner(n_warmup_steps=50))
    study.optimize(lambda trial: objective_catboost(trial, X, y, n_splits=n_splits_, n_repeats=n_repeats_, model=model_class, use_gpu=use_gpu), n_trials=n_trials)
    return study  # Return the study object

# Step 3: Saving Best Results and Models
def save_results(study, model_class, model_name):
    best_params_file = f"{model_name}_best_params.joblib"
    joblib.dump(study.best_params, best_params_file)
    print(f"Best parameters for {model_name} saved to {best_params_file}")

    verbose_file = f"{model_name}_optuna_verbose.log"
    with open(verbose_file, "w") as f:
        f.write(str(study.trials))
    print(f"Optuna verbose for {model_name} saved to {verbose_file}")

In [ ]:
# usage with XGBRegressor
cat_study = tune_hyperparameters(X, log_y, model_class=CatBoostRegressor, n_trials=101, n_splits_ = 3 ,n_repeats_=3, use_gpu=True)
save_results(cat_study, CatBoostRegressor, "CatBoost_ext")
cat_params = cat_study.best_params

[I 2024-12-12 19:11:51,789] A new study created in memory with name: no-name-20b61bd5-fc59-49d4-b064-9bcf33e3d6bf


0:	learn: 1.0942937	test: 1.0959597	best: 1.0959597 (0)	total: 55.4ms	remaining: 2m 18s
100:	learn: 1.0732499	test: 1.0751133	best: 1.0751133 (100)	total: 4.18s	remaining: 1m 39s
200:	learn: 1.0718021	test: 1.0741140	best: 1.0741140 (200)	total: 8.2s	remaining: 1m 33s
300:	learn: 1.0709775	test: 1.0737213	best: 1.0737213 (300)	total: 12s	remaining: 1m 27s
400:	learn: 1.0702517	test: 1.0734661	best: 1.0734661 (400)	total: 15.9s	remaining: 1m 23s
500:	learn: 1.0695737	test: 1.0732931	best: 1.0732931 (500)	total: 19.8s	remaining: 1m 18s
600:	learn: 1.0685728	test: 1.0728312	best: 1.0728312 (600)	total: 23.7s	remaining: 1m 14s
700:	learn: 1.0673791	test: 1.0722562	best: 1.0722540 (698)	total: 27.9s	remaining: 1m 11s
800:	learn: 1.0664158	test: 1.0719886	best: 1.0719860 (797)	total: 32s	remaining: 1m 7s
900:	learn: 1.0654992	test: 1.0717719	best: 1.0717711 (896)	total: 36s	remaining: 1m 3s
1000:	learn: 1.0646683	test: 1.0715839	best: 1.0715839 (999)	total: 40.2s	remaining: 1m
1100:	learn: 1

[I 2024-12-12 19:27:52,641] Trial 0 finished with value: 1.0699177857001105 and parameters: {'depth': 8, 'l2_leaf_reg': 0.014594085473136007, 'subsample ': 0.75, 'random_strength': 1.4825387145010973}. Best is trial 0 with value: 1.0699177857001105.


0:	learn: 1.0941498	test: 1.0958334	best: 1.0958334 (0)	total: 156ms	remaining: 6m 29s
100:	learn: 1.0665326	test: 1.0732521	best: 1.0732521 (100)	total: 13.1s	remaining: 5m 12s
200:	learn: 1.0579690	test: 1.0722933	best: 1.0722933 (200)	total: 26.3s	remaining: 5m 1s
300:	learn: 1.0476860	test: 1.0719895	best: 1.0719895 (300)	total: 39.4s	remaining: 4m 47s
400:	learn: 1.0380204	test: 1.0717974	best: 1.0717923 (384)	total: 52.4s	remaining: 4m 34s
500:	learn: 1.0280587	test: 1.0717992	best: 1.0717541 (448)	total: 1m 5s	remaining: 4m 21s
bestTest = 1.071754075
bestIteration = 448
Shrink model to first 449 iterations.
0:	learn: 1.0954400	test: 1.0932569	best: 1.0932569 (0)	total: 147ms	remaining: 6m 7s
100:	learn: 1.0678266	test: 1.0713042	best: 1.0713042 (100)	total: 13.2s	remaining: 5m 13s
200:	learn: 1.0580321	test: 1.0703879	best: 1.0703821 (195)	total: 26.5s	remaining: 5m 3s
300:	learn: 1.0475933	test: 1.0701235	best: 1.0701194 (299)	total: 39.6s	remaining: 4m 49s
400:	learn: 1.037934

[I 2024-12-12 19:41:40,747] Trial 1 finished with value: 1.0708524212754442 and parameters: {'depth': 13, 'l2_leaf_reg': 0.4894207717033368, 'subsample ': 0.675, 'random_strength': 0.18065512282671958}. Best is trial 0 with value: 1.0699177857001105.


...

0:	learn: 1.0942948	test: 1.0959604	best: 1.0959604 (0)	total: 49.7ms	remaining: 2m 4s
100:	learn: 1.0727410	test: 1.0744983	best: 1.0744983 (100)	total: 4.24s	remaining: 1m 40s
200:	learn: 1.0710623	test: 1.0732461	best: 1.0732461 (200)	total: 8.41s	remaining: 1m 36s
300:	learn: 1.0701167	test: 1.0727912	best: 1.0727912 (300)	total: 12.5s	remaining: 1m 31s
400:	learn: 1.0692670	test: 1.0724645	best: 1.0724645 (400)	total: 16.6s	remaining: 1m 27s
500:	learn: 1.0684580	test: 1.0721934	best: 1.0721934 (500)	total: 20.8s	remaining: 1m 22s
600:	learn: 1.0675188	test: 1.0718824	best: 1.0718824 (600)	total: 25s	remaining: 1m 18s
700:	learn: 1.0666125	test: 1.0716484	best: 1.0716450 (695)	total: 29.2s	remaining: 1m 14s
800:	learn: 1.0657659	test: 1.0714743	best: 1.0714743 (800)	total: 33.3s	remaining: 1m 10s
900:	learn: 1.0648910	test: 1.0713267	best: 1.0713267 (896)	total: 37.4s	remaining: 1m 6s
1000:	learn: 1.0641129	test: 1.0712235	best: 1.0712224 (991)	total: 41.4s	remaining: 1m 2s
1100:	

[I 2024-12-13 02:45:22,755] Trial 29 finished with value: 1.0698905180255285 and parameters: {'depth': 8, 'l2_leaf_reg': 0.0012638244219827354, 'subsample ': 0.75, 'random_strength': 0.49869034319778516}. Best is trial 7 with value: 1.069768965999698.


0:	learn: 1.0942474	test: 1.0959139	best: 1.0959139 (0)	total: 53.5ms	remaining: 2m 13s
100:	learn: 1.0720242	test: 1.0738959	best: 1.0738959 (100)	total: 4.82s	remaining: 1m 54s
200:	learn: 1.0699089	test: 1.0726438	best: 1.0726438 (200)	total: 9.62s	remaining: 1m 50s
300:	learn: 1.0685667	test: 1.0722628	best: 1.0722628 (300)	total: 14.3s	remaining: 1m 44s
400:	learn: 1.0672471	test: 1.0719054	best: 1.0719054 (400)	total: 19.1s	remaining: 1m 39s
500:	learn: 1.0660143	test: 1.0716637	best: 1.0716620 (499)	total: 23.8s	remaining: 1m 35s
600:	learn: 1.0647084	test: 1.0714463	best: 1.0714457 (599)	total: 28.6s	remaining: 1m 30s
700:	learn: 1.0633760	test: 1.0712315	best: 1.0712295 (698)	total: 33.4s	remaining: 1m 25s
800:	learn: 1.0620569	test: 1.0710923	best: 1.0710915 (799)	total: 38.2s	remaining: 1m 20s
900:	learn: 1.0607308	test: 1.0709558	best: 1.0709558 (900)	total: 42.9s	remaining: 1m 16s
1000:	learn: 1.0594563	test: 1.0708502	best: 1.0708502 (1000)	total: 47.8s	remaining: 1m 11s


[I 2024-12-13 02:59:48,654] Trial 30 finished with value: 1.0697489577569923 and parameters: {'depth': 9, 'l2_leaf_reg': 0.00041192996871388494, 'subsample ': 0.9, 'random_strength': 0.32197876413585336}. Best is trial 30 with value: 1.0697489577569923.


0:	learn: 1.0942474	test: 1.0959139	best: 1.0959139 (0)	total: 53.9ms	remaining: 2m 14s
100:	learn: 1.0720340	test: 1.0739083	best: 1.0739083 (100)	total: 4.95s	remaining: 1m 57s
200:	learn: 1.0699434	test: 1.0726328	best: 1.0726328 (200)	total: 9.76s	remaining: 1m 51s
300:	learn: 1.0686290	test: 1.0723005	best: 1.0723005 (300)	total: 14.5s	remaining: 1m 45s
400:	learn: 1.0672959	test: 1.0719882	best: 1.0719882 (400)	total: 19.2s	remaining: 1m 40s
500:	learn: 1.0659482	test: 1.0717142	best: 1.0717125 (499)	total: 23.9s	remaining: 1m 35s
600:	learn: 1.0646099	test: 1.0714519	best: 1.0714507 (599)	total: 28.7s	remaining: 1m 30s
700:	learn: 1.0632357	test: 1.0712468	best: 1.0712438 (698)	total: 33.5s	remaining: 1m 26s
800:	learn: 1.0619398	test: 1.0710862	best: 1.0710862 (800)	total: 38.3s	remaining: 1m 21s
900:	learn: 1.0606284	test: 1.0709698	best: 1.0709698 (900)	total: 43.1s	remaining: 1m 16s
1000:	learn: 1.0593980	test: 1.0708808	best: 1.0708787 (996)	total: 47.8s	remaining: 1m 11s
1

    parameters: {'depth': 9, 'l2_leaf_reg': 0.00041192996871388494, 'subsample ': 0.9, 'random_strength': 0.32197876413585336}. Best is trial 30 with value: 1.0697489577569923.


    parameters: {'depth': 9, 'l2_leaf_reg': 0.000311043236374346, 'subsample ': 0.9, 'random_strength': 0.6528523995521294}. Best is trial 7 with value: 1.069768965999698.

In [ ]:
print(cat_params)

In [ ]:
trial = cat_study.best_trial
print('MSE: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

* Experiment 1:

    - MSE: 1.0685007914142697
    - Best hyperparameters: {'num_leaves': 111, 'min_child_samples': 35, 'reg_alpha': 005448690915044739, 'reg_lambda': 0.016061276667668913, 'max_depth': 12, 'colsample_bytree': 0.8500000000000001, 'subsample': 0.75}

* Experiment 2:

    - MSE: 1.068389091315945
    - Best hyperparameters: {'reg_alpha': 0.4932, 'reg_lambda': 0.002076, 'max_depth': 21, 'colsample_bytree': 0.825, 'subsample': 0.75,  'num_leaves': 101, 'n_estimators': 2501,'learning_rate': 0.01, 'min_child_samples':  35}    

In [ ]:
fig = optuna.visualization.plot_optimization_history(cat_study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_param_importances(cat_study)
fig.show()

In [ ]:
#del xgb_study
gc.collect()

### 1.2 Fit Best Model:

**PARAMETERS**

In [ ]:
# Define a common random seed for reproducibility
RANDOM_SEED = 42
N_ESTIMATORS = 3000  # Number of estimators for the ensemble models
n_splits = 3
n_repeats = 3

params ={'reg_alpha': 0.4932, 'reg_lambda': 0.002076, 'max_depth': 21, 'colsample_bytree': 0.825, 'subsample': 0.75, 'num_leaves': 101, 'n_estimators': 2501, 'learning_rate': 0.01,
         'min_child_samples':35, 'random_state': 42, 'force_col_wise':True, 'verbose':0}

cat_cols = ['Gender', 'Marital Status','Number of Dependents','Occupation','Location','Policy Type','Previous Claims', 'Insurance Duration', 'Customer Feedback',
            'Smoking Status',  'Exercise Frequency', 'Property Type', 'Start_Year', 'Start_Month', 'Start_Day','Education Level']

num_cols = ['Age', 'Annual Income', 'Health Score', 'Vehicle Age', 'Credit Score', 'Policy Start Date', 'Premium_time_Mean']

cat_loc = [X.columns.get_loc(i) for i in cat_cols]

**DATA**

In [ ]:
df_subm_stack = df_subm.copy()

X = df_train_new.drop(columns=["Premium Amount"])
X_test = df_test_new.drop(columns=["Premium Amount"])
y = df_train_new["Premium Amount"]

log_y = np.log1p(y)

**FIT THE MODEL**

In [ ]:
kf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)
rmsle = []

# Initialize the Stack
df_subm_stack['Premium Amount'] = 0.0

i=0

oof_results_stack = pd.DataFrame(columns=list(range(n_splits*n_repeats)), index=X.index)

for idx_train, idx_valid in kf.split(X, log_y):

    print(f"Working on Fold {i}")

    # Split the data into training and validation sets for the current fold
    X_train, y_train = X.iloc[idx_train], log_y.iloc[idx_train].to_numpy().reshape(-1,1)
    X_valid, y_valid = X.iloc[idx_valid], log_y.iloc[idx_valid].to_numpy().reshape(-1,1)
    X_test_ = X_test.copy()

    scaler = StandardScaler()
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_valid[num_cols] = scaler.transform(X_valid[num_cols])
    X_test_[num_cols] = scaler.transform(X_test[num_cols])


    X_train = X_train.to_numpy()
    X_valid = X_valid.to_numpy()
    X_test_ = X_test_.to_numpy()


    if i >= 9:
        #print(stacking_model.get_params())
        # Fit the StackingRegressor
        LGBM_model = train_lgbm(params, X_train, y_train, cat_loc, use_gpu=False, X_val=X_valid, y_val=y_valid, es=101)
        obj = sio.dump(LGBM_model, f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S4E12/LGBM_base_{i}.skops")

    else:
        unknown_types = sio.get_untrusted_types(file=f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S4E12/LGBM_base_{i}.skops")
        LGBM_model = sio.load(f"/content/drive/MyDrive/Exercises/Studies_Structured_Data/Models/S4E12/LGBM_base_{i}.skops", trusted=unknown_types)

    stack_preds = np.exp(LGBM_model.predict(X_valid))

    oof_results_stack.iloc[idx_valid,i] = stack_preds.flatten()
    # Prepare the test data and make predictions
    error = root_mean_squared_log_error(np.exp(y_valid), stack_preds)

    rmsle.append(error)
    print(f"RMSLE fold {i}: {error}")

    # Aggregate the predictions across the 5 folds (averaging for ensemble)
    df_subm_stack['Premium Amount'] += np.exp(LGBM_model.predict(X_test_)) / (n_splits*n_repeats)
    i += 1

In [ ]:
np.mean(rmsle), np.std(rmsle)

### **1.3 Save Results:**

In [ ]:
(df_test_for,train_forecast_to_store) = store_results(for_test=df_subm_stack,for_train=oof_results_stack, model="LGBM", experiment=0)

In [ ]:
df_test_for.max(),train_forecast_to_store.max()